# CS224W Project Data Pre-processing
### If You Can Make it Through Gridlock Here, You Can Make it Through Gridlock Anywhere: Using Graph Neural Networks to Predict New York City Traffic

Charles Shaviro, Yi-Ting Tsai, Leda Liang



## Finalized data explanation:

* Time range: between 2024-09-01 to 2024-10-31 (2 months)
* Time interval: every 5 minutes
* Number of rows: 1066 = number of unique sensors (one sensor is one node in the graph)
* Number of columns: 17569 = 61 days * 24 hours * 12 (each hour has 12 5-min intervals) + 1 (the sensor column)
* Row names: The locations of sensors in the format of “40.608031,-74.13212”. The number before the comma is the latitude, the number after the comma is the longitude.
* Column names: Date and time in the format of “2024-09-01 00:00”, “2024-10-31 23:55”. The numbers before space represent the date. The numbers after space are “HH:MM”, in 24-hour format.
* Each entry: represents the average speed of the cars passing through the sensor during the 5-min interval.

In [ ]:
!pip install pandas
!pip install networkx


In [ ]:
import pandas as pd
import networkx as nx
import time as tm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = '/content/drive/MyDrive/CS224W_Final_Project/data/DOT_Traffic_Speeds_NBE_20241105.csv'
df = pd.read_csv(data_path)
print("The number of rows in the original df is "  + str(df.shape[0])) # 4203941

The number of rows in the original df is 4203941


In [ ]:
# create a new column for date
df['DATE'] = pd.to_datetime(df['DATA_AS_OF']).dt.strftime('%m/%d/%Y')
sorted_unique_dates = sorted(df['DATE'].unique())
print("The start date of df is "  + str(sorted_unique_dates[0]))
print("The end date of df is "  + str(sorted_unique_dates[-1]))

# get the 2-month data between '2024-09-01' to '2024-10-31'
df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y')
start_date = pd.to_datetime('2024-09-01')
end_date = pd.to_datetime('2024-10-31')
df = df[(df['DATE'] >= start_date) & (df['DATE'] <= end_date)]
print("The number of rows between " + str(start_date) + " and " + str(end_date) + " is "  + str(df.shape[0])) # 2141367


The start date of df is 07/05/2024
The end date of df is 11/05/2024
The number of rows between 2024-09-01 00:00:00 and 2024-10-31 00:00:00 is 2141367


In [ ]:
# create a new column for time (HR:MIN), in 24hr format
df['TIME'] = pd.to_datetime(df['DATA_AS_OF']).dt.strftime('%H:%M')
sorted_unique_times = sorted(df['TIME'].unique())
print("The first 10 times are "  + str(sorted_unique_times[0:10]))
print("The last 10 times are "  + str(sorted_unique_times[-11:-1]))

# Function to round the time to the nearest 5-minute interval
min = 5
def round_to_nearest_min(time_obj):
    minutes = time_obj.minute
    rounded_minutes = (minutes // min) * min
    return time_obj.replace(minute=rounded_minutes, second=0)

# Apply the rounding function to the 'TIME' column
df['TIME'] = pd.to_datetime(df['TIME'], format='%H:%M').dt.time
df['TIME_ROUNDED'] = df['TIME'].apply(round_to_nearest_min)

# check
sorted_unique_round_times = sorted(df['TIME_ROUNDED'].unique())
print("Length of rounded times "  + str(len(sorted_unique_round_times)))  # 24*12 = 288
print("The first 3 times are "  + str(sorted_unique_round_times[0:3]))
print("The last 3 times are "  + str(sorted_unique_round_times[-4:-1]))

<ipython-input-6-fb31c037ed16>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['TIME'] = pd.to_datetime(df['DATA_AS_OF']).dt.strftime('%H:%M')


The first 10 times are ['00:03', '00:04', '00:07', '00:08', '00:09', '00:13', '00:14', '00:17', '00:18', '00:19']
The last 10 times are ['23:39', '23:43', '23:44', '23:47', '23:48', '23:49', '23:53', '23:54', '23:57', '23:58']
Length of rounded times 288
The first 3 times are [datetime.time(0, 0), datetime.time(0, 5), datetime.time(0, 10)]
The last 3 times are [datetime.time(23, 40), datetime.time(23, 45), datetime.time(23, 50)]


In [ ]:
df

,ID,SPEED,TRAVEL_TIME,DATA_AS_OF,LINK_ID,LINK_POINTS,DATE,TIME,TIME_ROUNDED
159781,381,52.81,30,10/31/2024 11:58:09 PM,4616194,"40.608031,-74.13212 40.60759,-74.1409",2024-10-31,23:58:00,23:55:00
159782,377,52.81,74,10/31/2024 11:58:09 PM,4616195,"40.61486,-74.15738 40.60931,-74.15012 40.60846...",2024-10-31,23:58:00,23:55:00
159783,350,0.00,0,10/31/2024 11:58:09 PM,4616196,"40.63092,-74.14592 40.62975,-74.14593 40.62877...",2024-10-31,23:58:00,23:55:00
159784,378,54.68,49,10/31/2024 11:58:09 PM,4616197,"40.6210105,-74.168861 40.6207604,-74.168 40.61...",2024-10-31,23:58:00,23:55:00
159785,435,60.27,101,10/31/2024 11:58:09 PM,4616198,"40.62102,-74.168861 40.6212304,-74.16992 40.62...",2024-10-31,23:58:00,23:55:00
...,...,...,...,...,...,...,...,...,...
2301143,448,0.00,0,09/01/2024 12:03:03 AM,4620343,"40.77149,-73.99423 40.7719,-73.99401 40.77481,...",2024-09-01,00:03:00,00:00:00
2301144,417,0.00,0,09/01/2024 12:03:03 AM,4763649,"40.60414,-74.052411 40.60479,-74.050351 40.607...",2024-09-01,00:03:00,00:00:00
2301145,411,0.00,0,09/01/2024 12:03:03 AM,4763652,"40.6040405,-74.052321 40.6047,-74.050301 40.60...",2024-09-01,00:03:00,00:00:00
2301146,264,0.00,0,09/01/2024 12:03:03 AM,4763655,"40.6083804,-74.039301 40.6107105,-74.03202 40....",2024-09-01,00:03:00,00:00:00


In [ ]:
# Task: find all the sensors
# split each sensor (i.e. a ['LATITUDE', 'LONGITUDE'] pair) into a new row -> get the unique sensors
df['sensor'] = df['LINK_POINTS'].str.split() # don't specify split(' ') since there might be multiple spaces
print(df.shape)
df_explode = df.explode('sensor', ignore_index=True)  # in df_explode, each row is a data entry for a sensor
print(df_explode.shape)
df_unique_sensors = df_explode.drop_duplicates(subset='sensor', keep='first')
print(df_unique_sensors.shape)
print("The number of unique sensors between " + str(start_date) + " and " + str(end_date) + " is " + str(df_unique_sensors.shape[0])) # num of nodes = 1269

(2141367, 10)
(23026557, 10)
(1269, 10)
The number of unique sensors between 2024-09-01 00:00:00 and 2024-10-31 00:00:00 is 1269


In [ ]:
# create the finalized data matrix for our GNN, each entry is the average speed of cars that passed through the sensor in the 5-min interval
# nrow of data is # of unique sensors, ncol of data is 61*24*12 = 17568
data = df_unique_sensors[['sensor']]   # create new df

# Generate all the dates between start_date and end_date
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
sensors_list = list(df_unique_sensors['sensor'])

start_time = tm.time()

for date in date_range:
    curr_date = date.strftime('%Y-%m-%d')
    df_a_day = df_explode[df_explode['DATE'] == curr_date]

    for time in sorted_unique_round_times:
        df_a_time = df_a_day[df_a_day['TIME_ROUNDED'] == time]
        new_col = []

        for sensor in sensors_list:
            df_a_sensor = df_a_time[df_a_time['sensor'] == sensor]
            speed = df_a_sensor['SPEED'].mean()
            new_col.append(speed)

        # create new columns for each 5 mins
        curr_time = time.strftime('%H:%M')
        col_name = curr_date + ' ' + curr_time
        data[col_name] = new_col

        # print status
        if curr_time in ['00:00', '06:00', '12:00', '18:00']:
            print(col_name + " is done")

end_time = tm.time()
print("Time taken: " + str(end_time - start_time) + " seconds")

# save
data.to_csv('/content/drive/MyDrive/CS224W_Final_Project/processed_data/data.csv', index=False)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# load data file
data = pd.read_csv('/content/drive/MyDrive/CS224W_Final_Project/processed_data/data.csv')
data

,sensor,2024-09-01 00:00,2024-09-01 00:05,2024-09-01 00:10,2024-09-01 00:15,2024-09-01 00:20,2024-09-01 00:25,2024-09-01 00:30,2024-09-01 00:35,2024-09-01 00:40,...,2024-10-31 23:10,2024-10-31 23:15,2024-10-31 23:20,2024-10-31 23:25,2024-10-31 23:30,2024-10-31 23:35,2024-10-31 23:40,2024-10-31 23:45,2024-10-31 23:50,2024-10-31 23:55
0,"40.608031,-74.13212",50.95,50.33,48.46,54.68,53.43,50.33,47.22,51.57,55.3,...,49.7,50.33,48.46,49.08,47.22,52.81,42.25,49.08,52.81,52.81
1,"40.60759,-74.1409",50.95,50.33,48.46,54.68,53.43,50.33,47.22,51.57,55.3,...,49.7,50.33,48.46,49.08,47.22,52.81,42.25,49.08,52.81,52.81
2,"40.61486,-74.15738",50.95,50.33,48.46,54.68,53.43,50.33,47.22,51.57,55.3,...,49.7,50.33,48.46,49.08,47.22,52.81,42.25,49.08,52.81,52.81
3,"40.60931,-74.15012",50.95,50.33,48.46,54.68,53.43,50.33,47.22,51.57,55.3,...,49.7,50.33,48.46,49.08,47.22,52.81,42.25,49.08,52.81,52.81
4,"40.60846,-74.14847",50.95,50.33,48.46,54.68,53.43,50.33,47.22,51.57,55.3,...,49.7,50.33,48.46,49.08,47.22,52.81,42.25,49.08,52.81,52.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,"40.6115804,-74.031051",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1265,"40.6127904,-74.030031",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1266,"40.6137105,-74.02903",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1267,"40.614411,-74.028201",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
# further clean the data
# some problem cases: "40.73334,-",

# 1. remove the rows with all 0
data_clean = data[data.iloc[:, 1:].sum(axis=1) != 0]

# 2. remove the rows with no commas: invalid sensor (missing 'LONGITUDE' data)
# 3. remove the rows with two commas: errors in the data set, they forget to put a space in between
data_clean = data_clean[data_clean['sensor'].str.count(',') == 1]

# 4. remove the rows that ends with a "-"
data_clean = data_clean[~data_clean['sensor'].str.endswith('-', na=False)]

# 5. remove the rows that ends with a ","
data_clean = data_clean[~data_clean['sensor'].str.endswith(',', na=False)]

In [ ]:
data_clean

,sensor,2024-09-01 00:00,2024-09-01 00:05,2024-09-01 00:10,2024-09-01 00:15,2024-09-01 00:20,2024-09-01 00:25,2024-09-01 00:30,2024-09-01 00:35,2024-09-01 00:40,...,2024-10-31 23:10,2024-10-31 23:15,2024-10-31 23:20,2024-10-31 23:25,2024-10-31 23:30,2024-10-31 23:35,2024-10-31 23:40,2024-10-31 23:45,2024-10-31 23:50,2024-10-31 23:55
0,"40.608031,-74.13212",50.95,50.33,48.46,54.68,53.43,50.33,47.22,51.57,55.30,...,49.70,50.33,48.46,49.08,47.22,52.81,42.25,49.08,52.81,52.81
1,"40.60759,-74.1409",50.95,50.33,48.46,54.68,53.43,50.33,47.22,51.57,55.30,...,49.70,50.33,48.46,49.08,47.22,52.81,42.25,49.08,52.81,52.81
2,"40.61486,-74.15738",50.95,50.33,48.46,54.68,53.43,50.33,47.22,51.57,55.30,...,49.70,50.33,48.46,49.08,47.22,52.81,42.25,49.08,52.81,52.81
3,"40.60931,-74.15012",50.95,50.33,48.46,54.68,53.43,50.33,47.22,51.57,55.30,...,49.70,50.33,48.46,49.08,47.22,52.81,42.25,49.08,52.81,52.81
4,"40.60846,-74.14847",50.95,50.33,48.46,54.68,53.43,50.33,47.22,51.57,55.30,...,49.70,50.33,48.46,49.08,47.22,52.81,42.25,49.08,52.81,52.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,"40.8448604,-73.92471",22.36,22.36,19.88,21.12,26.09,39.76,44.73,45.36,45.36,...,16.15,16.77,16.77,16.77,17.39,17.39,16.15,11.18,9.94,7.45
1226,"40.84488,-73.92327",22.36,22.36,19.88,21.12,26.09,39.76,44.73,45.36,45.36,...,16.15,16.77,16.77,16.77,17.39,17.39,16.15,11.18,9.94,7.45
1227,"40.8451305,-73.91885",22.36,22.36,19.88,21.12,26.09,39.76,44.73,45.36,45.36,...,16.15,16.77,16.77,16.77,17.39,17.39,16.15,11.18,9.94,7.45
1228,"40.8452706,-73.913131",22.36,22.36,19.88,21.12,26.09,39.76,44.73,45.36,45.36,...,16.15,16.77,16.77,16.77,17.39,17.39,16.15,11.18,9.94,7.45


In [ ]:
# save to file
data_clean.to_csv('/content/drive/MyDrive/CS224W_Final_Project/processed_data/data_clean.csv', index=False)